# 台積電財報 RAG 系統專案報告 (整合 OCR 功能)

## 專案簡介
本專案旨在建立一個基於檢索增強生成 (Retrieval Augmented Generation, RAG) 技術的問答系統，能夠針對台積電的合併財務報告提供精準回答。為了解決 PDF 檔案中可能包含圖片形式的文字（如表格），本專案整合了 OCR (Optical Character Recognition) 技術，增強了系統從不同格式的文本中提取資訊的能力，進而改善問答的準確性。

## 數據來源
本專案使用的數據來源是從公開渠道獲取的**台積電 114 年及 113 年第一季合併財務報告**的 PDF 檔案 (`202501_2330_AI1_20250801_131448.pdf`)。


###掛載 Google Drive
本專案會將處理後的數據和向量索引存儲在 Google Drive 中，以便在不同 Colab 工作階段中重複使用。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 安裝必要套件和軟體

使用 PyMuPDF 和 pytesseract 結合進行 OCR，以提取包含圖片頁面的 PDF 文字，並將提取的文字用於改進 RAG 系統的檢索和回答能力。
安裝 `pytesseract` Python 函式庫以及 Tesseract OCR 引擎本身。


In [ ]:
# 安裝 Tesseract OCR 引擎
!sudo apt-get update
!sudo apt-get install tesseract-ocr -y

# 安裝 Tesseract 中文語言包
!sudo apt-get install tesseract-ocr-chi-tra -y # 繁體中文

# 安裝 pytesseract Python 函式庫
!pip install pytesseract

!pip install pymupdf

## 數據前處理
數據前處理是 RAG 系統的關鍵步驟，旨在將非結構化的 PDF 內容轉換為可供檢索模型使用的結構化文本塊。

**PDF 文本提取 (含 OCR)**:
    *   使用 **PyMuPDF** 庫進行基礎文本提取。
    *   為了處理圖片頁面或文字提取不足的頁面，結合 **pytesseract** 進行 OCR。
    *   修改了提取函式 `extract_text_from_pdf_pymupdf_with_ocr`，當單頁直接提取的文字少於預設閾值 (`text_threshold=50`) 時，會將該頁渲染為圖片並使用 Tesseract OCR 進行文字辨識（使用繁體中文語言包 `chi_tra`），再將 OCR 結果添加到總文本中。
    *   提取的文本被儲存到變數 `tsmc_financial_report_text_with_ocr`。

In [ ]:
import fitz # 導入 PyMuPDF 庫
import pytesseract
from PIL import Image # 導入 Pillow 庫的 Image 模組
import io # 導入 io 模組用於處理圖片數據

def extract_text_from_pdf_pymupdf_with_ocr(pdf_file_path, text_threshold=50):
    """
    使用 PyMuPDF 從 PDF 檔案中提取所有文字內容。
    如果單頁提取的文字少於 text_threshold，則嘗試使用 OCR 從圖片中提取文字。
    """
    text_content = ""
    try:
        doc = fitz.open(pdf_file_path) # 開啟 PDF 文件
        print(f"成功開啟 PDF: {pdf_file_path}, 共 {doc.page_count} 頁")

        # 嘗試提取每一頁的文字
        for page_num in range(doc.page_count):
            try:
                page = doc.load_page(page_num) # 載入頁面

                # 優先嘗試直接提取文字
                page_text = page.get_text()

                # 檢查提取的文字量，如果很少，嘗試 OCR
                # 移除空白字符後計算長度，忽略空頁或只有少量標點符號的頁面
                if len(page_text.strip()) < text_threshold:
                    print(f"頁面 {page_num + 1} 直接提取文字量少 ({len(page_text.strip())} < {text_threshold})，嘗試 OCR...")

                    # 將頁面渲染為圖片
                    # dpi 越高，圖片越清晰，OCR 效果可能更好，但也會佔用更多記憶體和時間
                    pix = page.get_pixmap(dpi=300)
                    img = Image.open(io.BytesIO(pix.tobytes("png"))) # 將 pixmap 轉換為 PIL Image

                    # 使用 Tesseract 進行 OCR
                    # 使用繁體中文語言包
                    ocr_text = pytesseract.image_to_string(img, lang='chi_tra')
                    print(f"頁面 {page_num + 1} OCR 提取文字量: {len(ocr_text.strip())}")

                    # 將 OCR 結果添加到總文本中
                    text_content += ocr_text + "\n--- Page Separator (OCR) ---\n"

                else:
                    # 如果直接提取的文字足夠多，使用直接提取的文字
                    text_content += page_text + "\n--- Page Separator (Text) ---\n"
                    print(f"已提取頁面 {page_num + 1} (直接提取)")

            except Exception as page_e:
                print(f"處理頁面 {page_num + 1} 時發生錯誤: {page_e}")
                text_content += f"\n--- Error extracting Page {page_num + 1} ---\n" # 標記提取失敗的頁面

        doc.close() # 關閉文件
        return text_content
    except Exception as e:
        print(f"開啟或處理 PDF 時發生錯誤: {e}")
        return None

pdf_file = "/content/drive/MyDrive/financial_rag_project/data financial_reports/202501_2330_AI1_20250801_131448.pdf"

# 呼叫函式並將提取的文字內容儲存到變數中
tsmc_financial_report_text_with_ocr = extract_text_from_pdf_pymupdf_with_ocr(pdf_file, text_threshold=50)

if tsmc_financial_report_text_with_ocr:
    print("\n文字內容已成功使用 PyMuPDF + OCR 提取並儲存至變數 'tsmc_financial_report_text_with_ocr'。")
else:
    print("未能提取 PDF 內容。請確認檔案是否存在且未損壞。")

In [ ]:
# 將提取的文本儲存到檔案以便察看結果
output_text_file = "/content/extracted_tsmc_text.txt"
with open(output_text_file, "w", encoding="utf-8") as f:
    f.write(tsmc_financial_report_text_with_ocr)
print(f"提取的文本已儲存至: {output_text_file}")

## 更新文字清洗和後續步驟

**文字清洗**:
    *   由於 OCR 結果可能包含雜訊、錯誤辨識字符或不規則格式，開發了 `clean_financial_text_with_ocr` 函式進行清洗。
    *   清洗步驟包括：
        *   移除頁碼標記（如 `- 1 -`）。
        *   移除反斜線字符。
        *   將多個換行符替換為單個換行符。
        *   將多個空白字符替換為單個空格，並移除首尾空白。
        *   移除 OCR 或文本提取過程中可能產生的頁面分隔符 (`--- Page Separator ---`) 和錯誤標記。
    *   清洗後的文本儲存在 `cleaned_tsmc_text_with_ocr` 變數中，並保存到檔案 `/content/drive/MyDrive/FinancialRAGData/cleaned_tsmc_text_with_ocr.txt`。


In [ ]:
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json
import os

def clean_financial_text_with_ocr(text):
    """
    Clean financial text extracted from PDF, potentially including OCR results.
    Removes excess whitespace, line breaks, page number markers, OCR specific noise, etc.
    """
    print(f"--- Cleaning Text ---")
    print(f"Initial text length: {len(text)}")

    # Remove page number markers like '- 1 -', '- 23 -', etc.
    text = re.sub(r'- \d+ -', '', text)
    print(f"After removing page numbers: {len(text)}")

    # Remove '\' characters (if present from tools)
    text = re.sub(r'\\', '', text)
    print(f"After removing backslashes: {len(text)}")

    # Replace multiple newline characters with a single newline
    text = re.sub(r'\n{2,}', '\n', text)
    print(f"After normalizing newlines: {len(text)}")

    # Replace multiple whitespace characters with a single space, then strip leading/trailing whitespace
    # Doing this early might help simplify subsequent steps
    text = re.sub(r'\s+', ' ', text).strip()
    print(f"After normalizing whitespace and strip: {len(text)}")

    # Remove "--- Page Separator ---" markers
    text = re.sub(r'--- Page Separator \(OCR\) ---', '', text)
    print(f"After removing OCR separator: {len(text)}")
    text = re.sub(r'--- Page Separator \(Text\) ---', '', text)
    print(f"After removing Text separator: {len(text)}")
    text = re.sub(r'--- Error extracting Page \d+ ---', '', text) # Remove error markers as well
    print(f"After removing error markers: {len(text)}")


    # After cleaning, re-normalize whitespace and strip again
    # This step is important after removing various patterns
    text = re.sub(r'\s+', ' ', text).strip()
    print(f"Final text length after re-normalizing whitespace and strip: {len(text)}")


    return text

if 'tsmc_financial_report_text_with_ocr' not in globals() or tsmc_financial_report_text_with_ocr is None or len(tsmc_financial_report_text_with_ocr.strip()) == 0:
    print("錯誤: 原始或包含 OCR 結果的 PDF 文本變數 'tsmc_financial_report_text_with_ocr' 不存在、為 None 或為空。")
    print("請先運行 PDF 提取儲存格並確認其輸出顯示成功提取到文字。")
    cleaned_tsmc_text_with_ocr = None # Ensure variable is set to None if source text is missing or empty
else:
    cleaned_tsmc_text_with_ocr = clean_financial_text_with_ocr(tsmc_financial_report_text_with_ocr)

    print("\n--- Cleaned Text with OCR Example ---")
    print(f"\nCleaned text length: {len(cleaned_tsmc_text_with_ocr)} characters")

    # --- 新增：保存清洗後的文本到檔案 ---
    output_dir = "/content/drive/MyDrive/FinancialRAGData" # 與 Streamlit 應用程式中設定的路徑一致
    cleaned_text_path = os.path.join(output_dir, "cleaned_tsmc_text_with_ocr.txt")
    os.makedirs(output_dir, exist_ok=True) # 確保目錄存在

    try:
        with open(cleaned_text_path, "w", encoding='utf-8') as f:
            f.write(cleaned_tsmc_text_with_ocr)
        print(f"\n清洗後的文本已保存至: {cleaned_text_path}")
    except Exception as e:
        print(f"\n保存清洗後的文本檔案失敗: {e}")

**文本切割 (Chunking)**:
    *   使用 LangChain 的 `RecursiveCharacterTextSplitter` 將清洗後的文本切割成較小的、有重疊的文本塊 (chunks)。
    *   參數設定為 `chunk_size=800` 和 `chunk_overlap=150`，這有助於保留上下文並確保關鍵資訊不會被分割在不同的塊中。
    *   為每個文本塊添加元數據，包括來源資訊和唯一的 `chunk_id`。
    *   切割後的文本塊儲存在 `all_processed_chunks_with_ocr` 列表中。

In [ ]:
if cleaned_tsmc_text_with_ocr is not None and len(cleaned_tsmc_text_with_ocr) > 0: # Only chunk if cleaned text is not empty
    def chunk_text_with_metadata(text, source_info, chunk_size=800, chunk_overlap=150):
        """
        使用 LangChain 的 RecursiveCharacterTextSplitter 進行文本切割，並為每個 chunk 添加元數據。
        """
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len, # 使用字元長度
            add_start_index=True, # 添加 chunk 在原始文本中的起始索引
        )

        chunks = text_splitter.create_documents([text], metadatas=[{"source": source_info}])

        formatted_chunks = []
        for i, chunk in enumerate(chunks):
            formatted_chunks.append({
                "text": chunk.page_content,
                "source": chunk.metadata["source"],
                "chunk_id": f"{source_info.replace(' ', '_')}_chunk_{i}" # 自定義一個唯一ID
            })
        return formatted_chunks

    tsmc_source_info = "台積電114年第1季合併財務報告 (含OCR)" # Update source info to reflect OCR inclusion

    # Execute text chunking with the cleaned OCR text
    all_processed_chunks_with_ocr = chunk_text_with_metadata(
        cleaned_tsmc_text_with_ocr,
        tsmc_source_info,
        chunk_size=800, # Keep the same chunk size and overlap as previous successful runs
        chunk_overlap=150
    )

    print(f"\n--- Text Chunking Example (with OCR) ---")
    print(f"Total chunks created: {len(all_processed_chunks_with_ocr)}")
    if len(all_processed_chunks_with_ocr) > 0: # Only try to print examples if chunks exist
        print("First chunk example:")
        print(all_processed_chunks_with_ocr[0])
        if len(all_processed_chunks_with_ocr) > 1: # Only print last if there's more than one
             print("\nLast chunk example:")
             print(all_processed_chunks_with_ocr[-1])
        else:
             print("\nOnly one chunk created.")

else:
    # If cleaning failed or resulted in empty text, set all_processed_chunks_with_ocr to None or empty list
    all_processed_chunks_with_ocr = [] # Set to empty list instead of None to avoid TypeErrors later
    print("\n文本清洗後為空，跳過文本切割。")

###文本嵌入與向量資料庫
建立向量索引
目的： 這是 RAG 系統的核心。我們需要將每個文本塊轉換成數值向量（嵌入），然後將這些向量存儲在一個高效能的向量資料庫 (FAISS) 中。這個資料庫能夠快速地搜尋與使用者問題最相似的文本塊。

使用的技術：

sentence-transformers： 我們使用 paraphrase-multilingual-MiniLM-L12-v2 這個多語言嵌入模型。它可以將中文文本轉換成高品質的語義向量。

faiss： Meta 開發的一個高效能相似度搜尋庫，非常適合用於處理大量的向量資料。

In [ ]:
!pip install faiss-cpu

In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
print("嵌入模型加載完成。")

def get_embedding(text):
    """生成文本的嵌入向量。"""
    # convert_to_tensor=False 返回 NumPy array
    return embedding_model.encode(text, convert_to_tensor=False)

# 步驟1：生成所有文本塊的嵌入
print("\n--- 生成所有文本塊的嵌入 ---")
all_embeddings = []
# document_store 將用於儲存原始文本和元數據，以便從 FAISS 索引恢復
document_store = {}

# 使用包含 OCR 結果的文本塊變數 all_processed_chunks_with_ocr
for i, chunk_info in enumerate(all_processed_chunks_with_ocr):
    embedding = get_embedding(chunk_info["text"])
    all_embeddings.append(embedding)
    # 將 chunk 的所有資訊儲存起來，索引就是它在 FAISS 中的 ID
    document_store[i] = chunk_info

all_embeddings_np = np.array(all_embeddings).astype('float32') # FAISS 需要 float32 類型

# 步驟2：建立 FAISS 索引
print("\n--- 建立 FAISS 索引 ---")
dimension = all_embeddings_np.shape[1] # 嵌入向量的維度
index = faiss.IndexFlatL2(dimension) # 使用 L2 距離進行搜索 (歐幾里得距離)
index.add(all_embeddings_np) # 將所有嵌入添加到索引中

print(f"FAISS 索引建立完成，共 {index.ntotal} 個向量 (文本塊)。")

# 步驟3：儲存 FAISS 索引和 document_store (方便下次直接加載，無需重複計算)

# 確保輸出目錄存在
output_dir = "/content/drive/MyDrive/FinancialRAGData" # 修改為您 Google Drive 中的路徑，使用一個目錄來存放檔案
os.makedirs(output_dir, exist_ok=True)

faiss_index_path = os.path.join(output_dir, "tsmc_financial_docs.faiss")
document_store_path = os.path.join(output_dir, "tsmc_document_store.json")

faiss.write_index(index, faiss_index_path)
with open(document_store_path, "w", encoding='utf-8') as f:
    json.dump(document_store, f, ensure_ascii=False, indent=4)

print(f"FAISS 索引已保存至: {faiss_index_path}")
print(f"文件資料庫已保存至: {document_store_path}")

###大型語言模型 (LLM) 整合
使用 Google Gemini Pro API
目的： 整合 Google Gemini Pro 模型，用於根據檢索到的上下文和使用者問題生成最終答案。我們選擇 Gemini API，是為了避免在 Colab CPU 環境下運行大型模型的效能瓶頸。

使用的技術：

google.generativeai： Google 提供的 SDK，用於輕鬆與 Gemini API 進行互動。

Colab Secrets： 用於安全地存儲 API 金鑰，避免將敏感資訊硬編碼在筆記本中。

In [ ]:
import google.generativeai as genai
from google.colab import userdata


API_KEY = userdata.get('GEMINI_API_KEY')
if not API_KEY:
    raise ValueError("請在 Colab 的 Secrets 中設定 'GEMINI_API_KEY'！")

genai.configure(api_key=API_KEY)

# --- 初始化 Gemini 模型 ---
print("\n--- 初始化 Google Gemini Pro 模型 ---")
llm_model = genai.GenerativeModel('models/gemini-1.5-flash-latest')
print("Gemini Pro 模型初始化完成。")

# --- 定義 RAG 的 Prompt 模板 ---
PROMPT_TEMPLATE = """
您是一位金融分析師，請根據提供的以下金融文件內容，簡潔、專業地回答問題。
如果文件內容沒有足夠的資訊來回答問題，請明確指出「文件內容不足以回答此問題」。

文件內容：
---
{context}
---

問題：
{question}

回答：
"""

def generate_answer(query_text):
    print(f"\n--- 處理問題 (使用 Gemini Pro): {query_text} ---")

    # 步驟1：檢索相關文件
    retrieved_docs_info = retrieve_documents(query_text)

    if not retrieved_docs_info:
        print("未檢索到相關文件。")
        return "對不起，我沒有找到相關的金融文件來回答這個問題。", []

    # 步驟2：組合上下文
    context_texts = [doc["text"] for doc in retrieved_docs_info]
    context = "\n\n".join(context_texts)
    print(f"--- 檢索到的上下文 ({len(context_texts)} 條) ---")
    print(context[:500] + "...") # 打印部分上下文預覽

    # 步驟3：格式化 Prompt
    full_prompt = PROMPT_TEMPLATE.format(context=context, question=query_text)

    # 步驟4：使用 Gemini API 生成答案
    try:
        # 調用 Gemini Pro 模型生成內容
        response = llm_model.generate_content(full_prompt)

        if hasattr(response, 'text'):
            generated_answer = response.text.strip()
        elif response.candidates: # 檢查是否有候選答案
            generated_answer = response.candidates[0].content.parts[0].text.strip()
        else:
            generated_answer = "Gemini 模型未能生成答案，可能因安全策略或其他內部錯誤。"
            if hasattr(response, 'prompt_feedback') and response.prompt_feedback.block_reason:
                print(f"Prompt 被阻擋，原因: {response.prompt_feedback.block_reason.name}")

        print(f"--- Gemini Pro 生成的答案 ---")
        print(generated_answer)

        return generated_answer, retrieved_docs_info

    except Exception as e:
        print(f"Gemini API 調用發生錯誤: {e}")
        return "生成答案時發生錯誤，請檢查 API 金鑰、網路連接或 Prompt 內容。", []

###建構檢索增強生成 (RAG) 管道與測試
整合所有功能
目的： 將之前建立的檢索功能和 LLM 生成功能組合成一個完整的 RAG 管道。這一步將是整個智慧問答系統的最終實現。

In [10]:
import re # 導入正規表達式模組

# 檢索功能 (包含混合檢索邏輯)
def retrieve_documents_with_ocr_index(query_text, top_k=10, cleaned_full_text=None):
    """
    根據查詢文本，從包含 OCR 數據的 FAISS 索引中檢索最相關的文件塊。
    對於特定的查詢，會嘗試基於關鍵詞的精確匹配。
    """

    # 優先檢查是否為股票代碼查詢，並嘗試關鍵詞搜索
    if "股票代碼" in query_text or "股票號碼" in query_text or "證券代碼" in query_text:
        print("檢測到股票代碼相關查詢，嘗試關鍵詞搜索...")
        if cleaned_full_text:
            # 使用正規表達式搜索 "股票代碼：" 後面的數字 (通常是4位)
            match = re.search(r"股票代碼[：:]\s*(\d+)", cleaned_full_text)
            if match:
                stock_code_snippet = match.group(0) # 獲取匹配到的完整片段，例如 "股票代碼： 2330"
                print(f"關鍵詞搜索找到匹配: {stock_code_snippet}")
                # 構建一個模擬的文本塊列表，包含找到的片段和來源資訊
                simulated_chunk = {
                    "text": stock_code_snippet,
                    "source": "文件內容 (股票代碼關鍵詞匹配)",
                    "chunk_id": "keyword_match_stock_code"
                }
                return [simulated_chunk]
            else:
                print("股票代碼關鍵詞搜索未找到匹配，繼續...")
        else:
            print("未提供完整的清洗後文本，無法執行股票代碼關鍵詞搜索，繼續...")

    # 檢查是否為營收相關查詢，並嘗試更複雜的關鍵詞搜索
    # 尋找包含「營收」或「合併營收」以及「114年」和「第1季」或「第一季」的模式
    revenue_pattern = re.compile(r".*?(合併營收|營收).*?(114年).*?(第1季|第一季).*?(\d{1,3}(,\d{3})*(\.\d+)?).*?", re.IGNORECASE)
    if ("營收" in query_text or "合併營收" in query_text) and ("114年" in query_text or "114 年" in query_text) and ("第一季" in query_text or "第1季" in query_text or "第 1 季" in query_text):
         print("檢測到114年第一季營收相關查詢，嘗試更複雜的關鍵詞搜索...")
         if cleaned_full_text:
             # 在清洗後的文本中搜索匹配的模式
             # 使用 finditer 找到所有匹配項
             matches = list(revenue_pattern.finditer(cleaned_full_text))
             if matches:
                 revenue_snippets = []
                 for match in matches:
                     # 提取匹配到的完整片段，或者根據需要提取特定組 (如數字)
                     snippet = match.group(0).strip()
                     print(f"營收關鍵詞搜索找到匹配: {snippet[:100]}...")
                     revenue_snippets.append({
                         "text": snippet,
                         "source": "文件內容 (營收關鍵詞匹配)",
                         "chunk_id": f"keyword_match_revenue_{matches.index(match)}"
                     })
                 return revenue_snippets
             else:
                 print("營收關鍵詞搜索未找到匹配，回退到語義搜索。")
         else:
             print("未提供完整的清洗後文本，無法執行營收關鍵詞搜索，回退到語義搜索。")


    # 如果不是上述特定查詢，或者關鍵詞搜索未找到結果，執行語義搜索
    print("執行語義搜索...")
    if 'get_embedding' not in globals():
        print("錯誤: get_embedding 函式未定義。請確保運行了加載嵌入模型的儲存格。")
        return []

    query_embedding = get_embedding(query_text).astype('float32')
    query_embedding = query_embedding.reshape(1, -1)

    if 'index_with_ocr' not in globals() or 'document_store_with_ocr' not in globals():
         print("錯誤: FAISS 索引或文件資料庫未載入。請確保運行了建立或載入索引的儲存格。")
         return []


    distances, indices = index_with_ocr.search(query_embedding, top_k)

    retrieved_docs_info = []
    for i in indices[0]:
        if i >= 0 and i < len(document_store_with_ocr):
            doc_info = document_store_with_ocr[i]
            retrieved_docs_info.append(doc_info)
        else:
             print(f"警告: 檢索到無效索引 {i}。")


    return retrieved_docs_info # 返回包含 text 和 source 的字典列表

def generate_answer_with_ocr_data(query_text):
    print(f"\n=======================================================")
    print(f"**問題:** {query_text}")
    print(f"--- 處理問題 (使用 Gemini Pro + OCR 數據): {query_text} ---")

    # 步驟1：檢索相關文件 (使用新的檢索函數，並傳入完整的清洗後文本)
    if 'cleaned_tsmc_text_with_ocr' not in globals():
        print("錯誤: cleaned_tsmc_text_with_ocr 未定義。請確保運行了文字清洗儲存格。")
        return "處理錯誤：無法找到清洗後的文本。", []

    retrieved_docs_info = retrieve_documents_with_ocr_index(
        query_text,
        top_k=10,
        cleaned_full_text=cleaned_tsmc_text_with_ocr
    )

    if not retrieved_docs_info:
        print("未檢索到相關文件。")
        print("\n**AI 的回答:**")
        print("對不起，我沒有找到相關的金融文件來回答這個問題。")
        print("**沒有找到直接相關的參考文件。**")
        print(f"=======================================================\n")
        return "對不起，我沒有找到相關的金融文件來回答這個問題。", []

    # 步驟2：組合上下文
    context_texts = [doc["text"] for doc in retrieved_docs_info]
    context = "\n\n".join(context_texts)
    print(f"--- 檢索到的上下文 ({len(retrieved_docs_info)} 條) ---")
    for i, text in enumerate(context_texts):
        print(f"Chunk {i+1} (Source: {retrieved_docs_info[i]['source']}): {text[:200]}...")
        if len(text) > 200:
            print("...")


    # 步驟3：格式化 Prompt (確保 PROMPT_TEMPLATE 在使用前被定義)
    # 將 PROMPT_TEMPLATE 的定義移到條件判斷之外，確保它總是作為局部變數被賦值
    PROMPT_TEMPLATE = """
您是一位金融分析師，請根據提供的以下金融文件內容，簡潔、專業地回答問題。
如果文件內容沒有足夠的資訊來回答問題，請明確指出「文件內容不足以回答此問題」。

文件內容：
---
{context}
---

問題：
{question}

回答：
"""


    full_prompt = PROMPT_TEMPLATE.format(context=context, question=query_text)

    # 步驟4：使用 Gemini API 生成答案 (確保 llm_model 已初始化)
    if 'llm_model' not in globals():
        print("Gemini model (llm_model) not initialized. Please run the initialization cell first.")
        print("\n**AI 的回答:**")
        print("Gemini 模型未初始化，無法生成答案。")
        print("**沒有找到直接相關的參考文件。**")
        print(f"=======================================================\n")
        return "Gemini 模型未初始化，無法生成答案。", []

    try:
        response = llm_model.generate_content(full_prompt)

        if hasattr(response, 'text'):
            generated_answer = response.text.strip()
        elif response.candidates:
            generated_answer = "".join([part.text for part in response.candidates[0].content.parts]).strip()
        else:
            generated_answer = "Gemini 模型未能生成答案，可能因安全策略或其他內部錯誤。"
            if hasattr(response, 'prompt_feedback') and response.prompt_feedback.block_reason:
                print(f"Prompt 被阻擋，原因: {response.prompt_feedback.block_reason.name}")

        print(f"\n**AI 的回答:**")
        print(generated_answer)

        if retrieved_docs_info:
            print("\n**參考來源 (部分文本與來源):**")
            for i, source_info in enumerate(retrieved_docs_info):
                print(f"  來源 {i+1}: {source_info['source']}")
                print(f"    內容摘要: {source_info['text'][:100]}...")
        else:
            print("\n**沒有找到直接相關的參考文件。**")

        print(f"=======================================================\n")


        return generated_answer, retrieved_docs_info

    except Exception as e:
        print(f"Gemini API 調用發生錯誤: {e}")
        print("\n**AI 的回答:**")
        print(f"生成答案時發生錯誤: {e}")
        print("**沒有找到直接相關的參考文件。**")
        print(f"=======================================================\n")
        return "生成答案時發生錯誤，請檢查 API 金鑰、網路連接或 Prompt 內容。", []

# 範例測試問題
# test_queries = [
#     "台積電114年第一季的合併營收是多少？",
#     "台積電的會計師核閱報告是由哪家會計師事務所出具的？",
#     "台積電在大陸的投資資訊有什麼？",
#     "台積電的股票代碼是什麼？",
#     "台積電有哪些子公司?"
#     "蘋果公司最近推出了什麼新產品？" 這個問題應該無法回答，因為不在文件內容中

## 安裝 Streamlit 和 ngrok
安裝 Streamlit 函式庫和 ngrok 工具（或 pyngrok）。

In [ ]:
# 安裝 Streamlit 和 pyngrok
!pip install streamlit pyngrok

import os
if not os.path.exists('/usr/local/bin/ngrok'):
    print("ngrok 執行檔未找到，正在下載並安裝...")
    !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
    !unzip ngrok-stable-linux-amd64.zip
    !sudo mv ngrok /usr/local/bin/
    print("ngrok 安裝完成。")
else:
    print("ngrok 執行檔已存在。")

# 驗證 ngrok 版本
!ngrok --version

### Streamlit 應用
為了提供一個友好的使用者介面來測試和展示 RAG 系統，本專案使用 Streamlit 建立了一個簡單的 Web 應用程式 (`app.py`)。

*   應用程式載入預先保存的嵌入模型、FAISS 索引和文件資料庫。
*   使用 `st.cache_resource` 快取模型和索引等資源，避免重複載入。
*   提供一個文本輸入框供用戶輸入問題。
*   點擊按鈕後，應用程式調用 RAG 邏輯（檢索相關文本並使用 Gemini 生成答案）。
*   在介面中顯示 AI 的回答以及作為參考的來源文本塊和來源資訊。
*   為了在 Colab 環境中運行 Streamlit 應用並通過公共 URL 訪問，使用了 **ngrok** (通過 `pyngrok` 庫) 建立 HTTP 通道。需要配置 ngrok Authtoken。Gemini API 金鑰通過臨時檔案安全地傳遞給 Streamlit 應用。


In [ ]:
%%writefile app.py
import streamlit as st
import json
import os
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
import re
import sys
import glob


st.title('TSMC 財報 RAG 系統')
st.write('請輸入關於**台積電114及113年第一季財報**的相關問題：')

# --- RAG 組件載入 (使用 Streamlit 快取) ---
@st.cache_resource
def load_rag_components():
    device = "cpu"
    st.write(f"模型將在設備: {device} 上加載和運行。")

    embedding_model = SentenceTransformer('bert-base-multilingual-cased', device=device)
    st.write("嵌入模型加載完成。")

    output_dir = "/content/drive/MyDrive/FinancialRAGData"
    faiss_index_path_with_ocr = os.path.join(output_dir, "tsmc_financial_docs_with_ocr.faiss")
    document_store_path_with_ocr = os.path.join(output_dir, "tsmc_document_store_with_ocr.json")
    cleaned_text_path = os.path.join(output_dir, "cleaned_tsmc_text_with_ocr.txt")

    # 載入 FAISS 索引
    st.write(f"--- 載入 FAISS 索引: {faiss_index_path_with_ocr} ---")
    if not os.path.exists(faiss_index_path_with_ocr):
        st.error(f"錯誤: FAISS 索引檔案不存在於 {faiss_index_path_with_ocr}")
        return None, None, None, None, None
    index_with_ocr = faiss.read_index(faiss_index_path_with_ocr)

    # 載入文件資料庫
    if not os.path.exists(document_store_path_with_ocr):
        st.error(f"錯誤: 文件資料庫檔案不存在於 {document_store_path_with_ocr}")
        return None, None, None, None, None

    with open(document_store_path_with_ocr, "r", encoding='utf-8') as f:
        document_store_with_ocr = {int(k): v for k, v in json.load(f).items()}
    st.write("文件資料庫載入完成。")

    # 載入清洗後的文本 (用於關鍵詞搜索的回退或輔助)
    if not os.path.exists(cleaned_text_path):
         st.error(f"錯誤: 清洗後的文本檔案不存在於 {cleaned_text_path}")
         st.warning("Consider saving the cleaned text to a file after the cleaning step in the Colab notebook.")
         cleaned_full_text = None
    else:
        with open(cleaned_text_path, "r", encoding='utf-8') as f:
            cleaned_full_text = f.read()


    # 初始化 Gemini 模型
    API_KEY = None
    temp_api_key_files = glob.glob(os.path.join(output_dir, ".gemini_api_key_*.tmp"))
    if temp_api_key_files:
        temp_api_key_path = temp_api_key_files[0]
        try:
            with open(temp_api_key_path, "r") as f:
                API_KEY = f.read().strip()
        except Exception as e:
             st.error(f"從臨時檔案讀取 GEMINI_API_KEY 失敗: {e}")


    if not API_KEY:
         API_KEY = os.getenv('GEMINI_API_KEY')


    if not API_KEY:
        st.error("請設定 'GEMINI_API_KEY' (在 Colab Secrets 並由筆記本寫入臨時檔案，或在部署環境中設定環境變數)！")
        llm_model = None
    else:
        try:
            genai.configure(api_key=API_KEY)
            llm_model = genai.GenerativeModel('models/gemini-1.5-flash-latest')
        except Exception as e:
            st.error(f"Gemini API 初始化失敗: {e}")
            if "'NoneType' object has no attribute 'kernel'" in str(e):
                 st.error("無法使用提供的 API 金鑰初始化 Gemini 模型。請確認金鑰有效。")
            llm_model = None


    return embedding_model, index_with_ocr, document_store_with_ocr, llm_model, cleaned_full_text

embedding_model, index_with_ocr, document_store_with_ocr, llm_model, cleaned_full_text = load_rag_components()

# 檢查核心組件是否成功載入 (嵌入模型、索引、文件庫和語言模型)
if embedding_model is None or index_with_ocr is None or document_store_with_ocr is None or llm_model is None:
    st.error("核心 RAG 組件載入失敗。請檢查 Colab Secrets、Google Drive 檔案路徑和檔案是否存在，並確保 API 金鑰有效。")

def get_embedding(text):
    if embedding_model is None:
         st.error("嵌入模型未載入，無法生成嵌入向量。")
         return None

    return embedding_model.encode(text, convert_to_tensor=False)


def retrieve_documents_optimized(query_text, top_k_semantic=10, top_k_keyword=5):
    """
    優化後的混合檢索函數：
    1. 先進行語義搜索，獲取 Top K 個相關文本塊。
    2. 對特定問題類型進行關鍵詞搜索 (可在語義結果中或全文)。
    3. 返回語義搜索結果和/或關鍵詞搜索結果。
    """
    st.info("檢索策略: 語義搜索優先，並對特定問題進行關鍵詞搜索。")

    # 步驟1：執行語義搜索
    query_embedding = get_embedding(query_text)
    if query_embedding is None:
        st.error("無法生成查詢嵌入向量，跳過檢索。")
        return []

    query_embedding = query_embedding.astype('float32')
    query_embedding = query_embedding.reshape(1, -1)

    # 使用載入的索引 index_with_ocr 進行搜索
    if index_with_ocr is None:
        st.error("FAISS 索引未載入，無法執行搜索。")
        return []

    distances, indices = index_with_ocr.search(query_embedding, top_k_semantic)

    semantic_docs_info = []
    if document_store_with_ocr is None:
        st.error("文件資料庫未載入，無法檢索文檔信息。")
        return []

    for i in indices[0]:
        if i >= 0 and i in document_store_with_ocr:
            doc_info = document_store_with_ocr[i]
            semantic_docs_info.append(doc_info)
        else:
             st.warning(f"警告: 語義檢索到無效索引 {i}。")


    # 步驟2：對特定問題類型進行關鍵詞搜索
    keyword_docs_info = []

    # 營收相關查詢的關鍵詞模式
    revenue_pattern = re.compile(r".*?(合併營收|營收).*?(114年).*?(第1季|第一季).*?(\d{1,3}(,\d{3})*(\.\d+)?).*?", re.IGNORECASE)

    # 股票代碼相關查詢的關鍵詞模式 (更簡單，只找數字)
    stock_code_pattern = re.compile(r"股票代碼[：:]\s*(\d+)", re.IGNORECASE)

    # 新增：會計師事務所相關查詢的關鍵詞模式
    # 搜索包含 "會計師" 和 "事務所" 的片段
    accounting_firm_pattern = re.compile(r".*?會計師.*?事務所.*?", re.IGNORECASE)


    # 判斷是否需要執行關鍵詞搜索，以及針對哪種模式
    target_pattern = None
    pattern_type = None

    if ("營收" in query_text or "合併營收" in query_text) and ("114年" in query_text or "114 年" in query_text) and ("第一季" in query_text or "第1季" in query_text or "第 1 季" in query_text):
        st.info("檢測到114年第一季營收相關查詢，嘗試關鍵詞搜索...")
        target_pattern = revenue_pattern
        pattern_type = "revenue"
        search_in_semantic_results = True
        search_in_full_text = False
    elif "股票代碼" in query_text or "股票號碼" in query_text or "證券代碼" in query_text:
        st.info("檢測到股票代碼相關查詢，嘗試關鍵詞搜索...")
        target_pattern = stock_code_pattern
        pattern_type = "stock_code"
        search_in_semantic_results = False
        search_in_full_text = True
    # 新增針對會計師事務所的判斷
    elif ("會計師" in query_text or "事務所" in query_text or "核閱報告" in query_text or "查核報告" in query_text):
        st.info("檢測到會計師事務所相關查詢，嘗試關鍵詞搜索...")
        target_pattern = accounting_firm_pattern
        pattern_type = "accounting_firm"
        search_in_semantic_results = False
        search_in_full_text = True


    if target_pattern:
        if search_in_full_text and cleaned_full_text:
            st.info("在清洗後的全文中執行關鍵詞搜索...")
            matches = list(target_pattern.finditer(cleaned_full_text))
            for match in matches:
                snippet = match.group(0).strip()
                start_index = match.start()
                end_index = match.end()
                context_start = max(0, start_index - 100)
                context_end = min(len(cleaned_full_text), end_index + 100)
                snippet_with_context = cleaned_full_text[context_start:context_end].strip()

                keyword_docs_info.append({
                    "text": snippet_with_context,
                    "source": f"關鍵詞匹配 ({pattern_type}) 全文搜索 (位置: {start_index}-{end_index})",
                    "chunk_id": f"keyword_match_{pattern_type}_full_{matches.index(match)}"
                })
                if len(keyword_docs_info) >= top_k_keyword:
                    break

            if not keyword_docs_info:
                 st.info("全文關鍵詞搜索未找到匹配，回退到語義搜索。")
                 final_retrieved_docs = semantic_docs_info
            else:
                 st.info(f"找到 {len(keyword_docs_info)} 條全文關鍵詞匹配結果。")
                 final_retrieved_docs = keyword_docs_info

        elif search_in_semantic_results:
            st.info("在語義搜索結果中執行關鍵詞搜索...")
            for i, doc_info in enumerate(semantic_docs_info):
                chunk_text = doc_info["text"]
                matches = list(target_pattern.finditer(chunk_text))
                for match in matches:
                     snippet = match.group(0).strip()
                     is_duplicate = any(snippet in existing_doc["text"] for existing_doc in keyword_docs_info)
                     if not is_duplicate:
                        keyword_docs_info.append({
                            "text": snippet,
                            "source": f"關鍵詞匹配 ({pattern_type}) from semantic chunk {i+1}",
                            "chunk_id": f"keyword_match_{pattern_type}_chunk_{i}_match_{matches.index(match)}"
                        })
                        if len(keyword_docs_info) >= top_k_keyword:
                            break

                if len(keyword_docs_info) >= top_k_keyword:
                     break

            if not keyword_docs_info:
                 st.info("語義結果中的關鍵詞搜索未找到匹配，返回語義搜索結果。")
                 final_retrieved_docs = semantic_docs_info #
            else:
                 st.info(f"找到 {len(keyword_docs_info)} 條語義結果中的關鍵詞匹配結果。")
                 final_retrieved_docs = keyword_docs_info[:]
                 for s_doc in semantic_docs_info:
                      is_covered_by_keyword = any(s_doc["text"] in k_doc["text"] for k_doc in keyword_docs_info)
                      if not is_covered_by_keyword:
                          is_duplicate = any(s_doc["chunk_id"] == f_doc.get("chunk_id") for f_doc in final_retrieved_docs)
                          if not is_duplicate:
                             final_retrieved_docs.append(s_doc)



        else:
             st.info("未設定關鍵詞搜索模式，返回語義搜索結果。")
             final_retrieved_docs = semantic_docs_info

    else:
        st.info("未偵測到特定關鍵詞模式，執行純語義搜索。")
        final_retrieved_docs = semantic_docs_info


    return final_retrieved_docs

def generate_answer_streamlit(query_text):
    """
    使用檢索到的文件生成答案。
    Assumes llm_model is available (cached).
    """
    # 檢查 LLM 模型是否成功載入
    if llm_model is None:
        st.error("LLM 模型未成功初始化，無法生成答案。請檢查 API 金鑰和載入過程。")
        return "無法生成答案，因為語言模型未成功初始化。", []


    # 步驟1：檢索相關文件 (使用優化後的混合檢索函數)
    retrieved_docs_info = retrieve_documents_optimized(
        query_text,
        top_k_semantic=10, # 語義搜索取前 10 個塊
        top_k_keyword=5 # 關鍵詞搜索最多取前 5 個匹配項
    )

    if not retrieved_docs_info:
        return "對不起，我沒有找到相關的金融文件來回答這個問題。", []

    # 步驟2：組合上下文
    # 為了避免上下文過長，我們可以限制用於生成答案的文本塊數量或總字元數
    max_context_chunks = 10
    context_texts = [doc["text"] for doc in retrieved_docs_info[:max_context_chunks]]
    context = "\n\n".join(context_texts)

    # 步驟3：格式化 Prompt
    PROMPT_TEMPLATE = """
您是一位金融分析師，請根據提供的以下金融文件內容，簡潔、專業地回答問題。
如果文件內容沒有足夠的資訊來回答問題，請明確指出「文件內容不足以回答此問題」。

文件內容：
---
{context}
---

問題：
{question}

回答：
"""
    full_prompt = PROMPT_TEMPLATE.format(context=context, question=query_text)

    # 步驟4：使用 Gemini API 生成答案
    try:
        response = llm_model.generate_content(full_prompt)

        if hasattr(response, 'text'):
            generated_answer = response.text.strip()
        elif response.candidates:
            generated_answer = "".join([part.text for part in response.candidates[0].content.parts]).strip()
        else:
            generated_answer = "Gemini 模型未能生成答案，可能因安全策略或其他內部錯誤。"
            if hasattr(response, 'prompt_feedback') and response.prompt_feedback.block_reason:
                st.warning(f"Prompt 被阻擋，原因: {response.prompt_feedback.block_reason.name}")

        return generated_answer, retrieved_docs_info

    except Exception as e:
        st.error(f"生成答案時發生錯誤: {e}")
        return "生成答案時發生錯誤，請檢查 API 金鑰、網路連接或 Prompt 內容。", []


query = st.text_input("請輸入您的問題:")

if st.button("獲取答案"):
    if query:
        if embedding_model is None or index_with_ocr is None or document_store_with_ocr is None or llm_model is None:
             st.error("RAG 系統尚未準備好。請檢查上述載入錯誤信息並解決問題。")
        else:
            with st.spinner("正在尋找答案..."):
                answer, sources = generate_answer_streamlit(query)

            st.subheader("AI 的回答:")
            st.write(answer)

            if sources:
                st.subheader(f"參考來源 ({len(sources)} 條):")
                max_sources_to_display = 10
                for i, source_info in enumerate(sources[:max_sources_to_display]):
                    st.write(f"**來源 {i+1}:** {source_info.get('source', '未知來源')}")
                    source_text_preview = source_info.get('text', '無法獲取文本內容')
                    st.write(f"內容摘要: {source_text_preview[:200]}...")
                    if len(source_text_preview) > 200:
                        st.write("...")
                if len(sources) > max_sources_to_display:
                    st.write(f"... 還有 {len(sources) - max_sources_to_display} 條來源未顯示。")

            else:
                st.info("沒有找到直接相關的參考文件。")
    else:
        st.warning("請輸入您的問題。")

## 運行包含 RAG 系統的 Streamlit 應用程式
在 Colab 中以後台方式運行 Streamlit 應用程式，並使用 ngrok 建立公共 URL。

In [ ]:
# 使用 pyngrok 建立 ngrok 通道並運行 Streamlit
from pyngrok import ngrok
import time
from google.colab import userdata # 導入 userdata 模組來讀取 secrets
import os # 導入 os 模組來檢查檔案是否存在和設定環境變數
import uuid # 導入 uuid 模組來生成唯一的臨時檔案名

# 從 Colab Secrets 中讀取 GEMINI_API_KEY
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    if not GEMINI_API_KEY:
        print("\nGEMINI_API_KEY not found in Colab Secrets. Please set it up.")
        raise ValueError("GEMINI_API_KEY not set in Colab Secrets.")
    else:
        print("GEMINI_API_KEY loaded from Secrets.")

except Exception as e:
    print(f"\nError retrieving GEMINI_API_KEY from Secrets: {e}")
    print("無法獲取 GEMINI_API_KEY，請檢查 Colab Secrets。")
    raise

# --- 將 API 金鑰寫入臨時檔案 ---
temp_api_key_filename = f".gemini_api_key_{uuid.uuid4().hex}.tmp"
output_dir = "/content/drive/MyDrive/FinancialRAGData"
temp_api_key_path = os.path.join(output_dir, temp_api_key_filename)
os.makedirs(output_dir, exist_ok=True)

try:
    with open(temp_api_key_path, "w") as f:
        f.write(GEMINI_API_KEY)
    print(f"GEMINI_API_KEY 已寫入臨時檔案: {temp_api_key_path}")

except Exception as e:
    print(f"\n錯誤：無法寫入臨時 API 金鑰檔案: {e}")
    raise

# --- 繼續 ngrok 和 Streamlit 啟動邏輯 ---

# 從 Colab Secrets 中讀取 ngrok Authtoken
try:
    NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    if not NGROK_AUTH_TOKEN:
        print("\nNGROK_AUTH_TOKEN not found in Colab Secrets. Skipping ngrok tunnel setup.")
        public_url = "NGROK_AUTH_TOKEN not set."
    else:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        print("ngrok Authtoken configured.")

        # 檢查 app.py 檔案是否存在
        if not os.path.exists("app.py"):
            print("\nError: app.py not found. Please ensure you have created the app.py file.")
            public_url = "app.py not found."
        else:
            # 啟動 Streamlit 應用程式在背景運行
            print("Starting Streamlit app in background...")
            !nohup streamlit run app.py --server.port 8501 > streamlit.log 2>&1 &

            # 等待 Streamlit 服務器啟動
            print("Waiting for Streamlit server to start...")
            time.sleep(20) # 顯著增加等待時間，給 Streamlit 更多時間啟動和載入 RAG 組件

            # 建立 ngrok http 通道到 Streamlit 運行的端口 (8501)
            print("Attempting to establish ngrok tunnel...")
            try:
                public_url = ngrok.connect(addr="8501", proto="http")
                print(f"\nStreamlit 應用程式正在運行，請透過以下 URL 訪問:")
                print(public_url)

                print("\nKeeping the notebook cell alive. Press stop to terminate.")
                time.sleep(3600)

            except Exception as e:
                print(f"\nError establishing ngrok tunnel: {e}")
                public_url = "Failed to establish ngrok tunnel. See error message above."

except Exception as e:
    # 如果在設置 GEMINI_API_KEY 或 ngrok Authtoken 階段出錯，或者在啟動Streamlit/ngrok時出錯，這裡會捕獲並打印
    print(f"\nAn error occurred during setup or Streamlit/ngrok launch: {e}")
    if 'public_url' not in locals():
         public_url = "An unexpected error occurred during launch."

finally:
    # --- 嘗試清理臨時檔案 ---
    if 'temp_api_key_path' in locals() and os.path.exists(temp_api_key_path):
        try:
            os.remove(temp_api_key_path)
            print(f"臨時 API 金鑰檔案已清理: {temp_api_key_path}")
        except Exception as e:
            print(f"\n警告：無法清理臨時 API 金鑰檔案: {e}")


print(f"\nFinal public_url status: {public_url}")

## 模型選擇與訓練
本專案使用了嵌入模型和向量資料庫來實現高效的文本檢索。

1.  **嵌入模型 (Embedding Model)**:
    *   使用 `sentence-transformers` 函式庫載入預訓練的嵌入模型。
    *   筆記本中嘗試並最終用於 Streamlit 應用的模型是 `bert-base-multilingual-cased`。這個模型支援多國語言，對於處理包含中文的金融文本是合適的選擇。
    *   `get_embedding` 函式用於將文本塊或查詢轉換為高維度向量。

2.  **向量資料庫 (FAISS Index)**:
    *   使用 **FAISS (Facebook AI Similarity Search)** 庫來建立和管理文本塊的嵌入向量索引。
    *   選擇了 `IndexFlatL2` 索引類型，使用 L2 距離（歐幾里得距離）來衡量向量之間的相似性。
    *   將所有文本塊的嵌入向量添加到 FAISS 索引中。
    *   建立了一個 `document_store` 字典，用於儲存每個索引對應的原始文本內容和元數據。
    *   將 FAISS 索引 (`tsmc_financial_docs.faiss`) 和 `document_store` (`tsmc_document_store.json`) 保存到 Google Drive 中，以便 Streamlit 應用程式載入。

## 結果與分析
通過整合 OCR 功能，本專案旨在提高 RAG 系統在處理包含圖像化文字的 PDF 檔案時的檢索準確性，進而提升問答效果。

*   **測試問題**: 筆記本中使用了多個測試問題來評估系統，包括：
    *   「台積電114年第一季的合併營收是多少？」
    *   「台積電的會計師核閱報告是由哪家會計師事務所出具的？」
    *   「台積電在大陸的投資資訊有什麼？」
    *   「台積電的股票代碼是什麼？」
    *   「蘋果公司最近推出了什麼新產品？」 (用於測試系統處理無關問題的能力)

*   **檢索策略改進**: 實現了混合檢索策略 `retrieve_documents_optimized`，結合了語義搜索（使用嵌入向量和 FAISS）和針對特定問題類型（如營收、股票代碼、會計師事務所）的關鍵詞匹配。關鍵詞匹配優先，這對於精確檢索特定事實資訊（如準確的營收數字或股票代碼）尤其有效，即使在語義搜索效果不佳的情況下也能提供相關依據。

*   **回答生成**: 使用 Google 的 **Gemini 1.5 Flash** 模型作為語言模型 (`llm_model`)。將檢索到的最相關文本塊作為上下文，與用戶查詢一起格式化成 Prompt 模板，提交給 Gemini 模型生成答案。Prompt 模板指示模型扮演金融分析師的角色，並在資訊不足時明確說明。

*   **效果分析**: 整合 OCR 後，系統能夠從原本可能無法提取文字的圖片頁面（如包含表格的頁面，筆記本輸出顯示頁面 3-10 進行了 OCR）中獲取資訊。這對於包含關鍵財務數據的表格尤為重要。混合檢索策略進一步提高了對特定問題的檢索精確度。測試結果應顯示系統能夠根據文件內容正確回答大多數問題，並在沒有相關資訊時給出適當的回應。與未整合 OCR 的系統相比，預期在處理包含大量圖片化數據的財報時表現會有顯著提升。

## 討論模型在實際應用中的潛力
這個整合 OCR 的 RAG 系統在金融領域，特別是處理 PDF 格式的財務報告方面具有顯著潛力：

*   **優勢**:
    *   能夠處理包含圖片和表格的複雜 PDF 格式，克服傳統文本提取的限制。
    *   提供基於具體文檔內容的回答，減少大型語言模型幻覺的可能性。
    *   混合檢索策略結合語義和關鍵詞匹配，提高了檢索的靈活性和準確性。
    *   提供參考來源，增強答案的可信度。
    *   Streamlit 介面使得系統易於使用和展示。

*   **潛在限制**:
    *   OCR 的準確性受圖片品質、字體、佈局複雜度影響，錯誤的 OCR 結果會直接影響下游檢索和生成。
    *   大型文件可能導致 Chunking 和嵌入過程計算成本較高，且單個 Prompt 的上下文長度有限。
    *   關鍵詞搜索的模式需要針對特定資訊手工編寫和調整。
    *   對於需要跨越多個文本塊才能獲取的複雜資訊，RAG 系統的表現可能受限。
    *   依賴外部 API (如 Gemini)，存在成本和穩定性問題。

*   **未來改進方向**:
    *   使用更先進的 OCR 模型或專門針對表格結構的提取工具。
    *   探索更有效的 Chunking 策略，例如基於語義段落或文件結構的切割。
    *   嘗試不同或更高性能的嵌入模型，特別是針對金融領域優化的模型。
    *   研究更複雜的檢索策略，例如重新排序 (Re-ranking) 或基於圖的檢索。
    *   整合更多金融數據源或歷史財報，擴展知識庫。
    *   實現更細粒度的來源引用，精確指出答案來自文檔的哪一部分。

## 結論與未來工作
本專案成功地將 OCR 技術整合到基於 PyMuPDF、Sentence-Transformers、FAISS 和 Gemini 的 RAG 系統中，提高了系統處理包含圖片化文字的 PDF 財務報告的能力。建立的 Streamlit 應用程式提供了一個可互動的介面來測試和展示系統。

未來工作應著重於進一步提升 OCR 準確性、優化文本處理和檢索策略，以及探索更豐富的數據源，從而構建一個更強大的金融 RAG 系統，更好地服務於金融分析和資訊查詢需求。